<a href="https://colab.research.google.com/github/mc-friday/hanghaeAI/blob/main/%5B4%EC%A3%BC%EC%B0%A8%5D%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [4주차] 기본과제: HuggingFace로 두 문장의 논리적 모순 분류하기

In [1]:
!pip install transformers datasets evaluate accelerate scikit-learn sacrebleu sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 re

In [ ]:
import time
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

## 1. [MY_CODE] 데이터셋 로드

- **과제 요구사항:**
    - load_dataset("nyu-mll/glue", "mnli") 로 dataset을 불러옵니다
    - 학습 때는 train split만 활용하셔야 합니다.
    - 나머지 split은 사용불가입니다.
    - Validation data가 필요한 경우, train split에서 가져오셔야 합니다
    - validation_matched에 대한 성능을 출력하고, 50%를 넘기셔야 합니다.

In [1]:
print("[LOG] Loading dataset...")
dataset = load_dataset("nyu-mll/glue", "mnli") # [과제적용]
train_split = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_data = train_split["train"]  # [과제적용] 학습용 데이터
validation_data = train_split["test"] # [과제적용] Validation 데이터 (train split에서 분리)
validation_matched_data = dataset["validation_matched"]  # [과제적용] validation_matched에 대한 성능을 출력에 사용

[LOG] Loading dataset...


NameError: name 'load_dataset' is not defined

## 2. [MY_CODE] DistilBERT 토크나이저 로드 및 데이터 전처리

- **입력:** premise에 해당하는 문장과 hypothesis에 해당하는 문장 두 개가 입력으로 들어옵니다.


In [ ]:
print("[LOG] Tokenizing dataset...")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):

    # premise와 hypothesis를 입력으로 처리
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        max_length=128
    )

train_data = train_data.map(preprocess_function, batched=True)  # 학습 데이터 전처리
validation_data = validation_data.map(preprocess_function, batched=True)  # Validation 데이터 전처리

# 데이터셋에서 메모리 낭비를 줄이기 위해 불필요한 열 제거
train_data = train_data.remove_columns(["premise", "hypothesis"])
validation_data = validation_data.remove_columns(["premise", "hypothesis"])

## 3. [MY_CODE] 모델 정의

- **출력:** 분류 문제로, 두 문장이 들어왔을 때 다음 세 가지를 예측하시면 됩니다.
    - **Entailment:** 두 문장에 논리적 모순이 없습니다.
    - **Neutral:** 두 문장은 논리적으로 관련이 없습니다.
    - **Contradiction:** 두 문장 사이에 논리적 모순이 존재합니다.

In [ ]:
print("[LOG] Loading model...")
# DistilBERT 모델을 로드하며, num_labels=3으로 세 가지 클래스를 예측합니다 (Entailment, Neutral, Contradiction)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
model

## 4. [MY_CODE] 평가 지표 정의

In [ ]:
def compute_metrics(pred):
    # 평가 지표는 정확도, F1 점수, 정밀도, 재현율로 구성됩니다.
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

## 5. [MY_CODE] 하이퍼파라미터 설정

In [ ]:
print("[LOG] Setting up training arguments...")
training_args = TrainingArguments(
    output_dir="./mnli_model",
    evaluation_strategy="epoch",  # [한글 주석] 매 Epoch마다 평가 수행
    save_strategy="epoch",  # [한글 주석] 매 Epoch마다 모델 저장
    learning_rate=5e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    logging_strategy="epoch",
    load_best_model_at_end=True,  # [한글 주석] 가장 성능이 좋은 모델을 마지막에 로드
    save_total_limit=2,  # [한글 주석] 저장할 체크포인트 개수 제한
    seed=42,
)

## 6. [MY_CODE] Trainer 설정

In [ ]:
print("[LOG] Setting up Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,  # [과제적용] train split만 학습에 사용
    eval_dataset=validation_data,  # [과제적용] Validation data는 train split에서 분리해 사용
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

## 7. [MY_CODE] 시각화 함수 정의

In [ ]:
def plot_metric(metric_values, metric_name, target=None):
    plt.figure()
    plt.plot(range(1, len(metric_values) + 1), metric_values, marker="o", label=metric_name)
    if target:
        plt.axhline(y=target, color="r", linestyle="--", label=f"Target ({target})")
    plt.title(f"{metric_name} per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel(metric_name)
    plt.legend()
    plt.grid()
    plt.show()


## 8. 학습

In [2]:
print("[LOG] Starting training...")
trainer.train()

# 학습 과정 시각화
history = trainer.state.log_history
losses = [log["loss"] for log in history if "loss" in log]
eval_accuracies = [log["eval_accuracy"] for log in history if "eval_accuracy" in log]
eval_f1_scores = [log["eval_f1"] for log in history if "eval_f1" in log]

# 시각화 실행
plot_metric(losses, "Loss")
plot_metric(eval_accuracies, "Accuracy", target=0.5)
plot_metric(eval_f1_scores, "F1 Score")

[LOG] Starting training...


NameError: name 'trainer' is not defined

## 9. [MY_CODE] Validation Matched 데이터 평가

- **validation_matched에 대한 성능을 출력하고, 50%를 넘겨야함**

In [ ]:
print("[LOG] Evaluating validation_matched dataset...")
results = trainer.evaluate(validation_data)  # [과제적용] Validation Matched 데이터 평가
validation_accuracy = results['eval_accuracy']  # Validation Matched 정확도 저장
print(f"Validation Matched Accuracy: {validation_accuracy:.2f}")

# 정확도가 50% 이상인지 확인 및 로그 출력
if validation_accuracy >= 0.5:
    print(f"[LOG] Validation Matched Accuracy meets the target: {validation_accuracy:.2f}")
else:
    print(f"[LOG] Validation Matched Accuracy below target: {validation_accuracy:.2f}")

# 8. 정확도 그래프 시각화
def plot_validation_accuracy(validation_accuracy, target=0.5):
    plt.figure()
    plt.bar(["Validation Matched Accuracy"], [validation_accuracy], color="blue", alpha=0.7, label="Accuracy")
    plt.axhline(y=target, color="red", linestyle="--", label=f"Target ({target * 100:.0f}%)")  # 50% 기준선
    plt.ylim(0, 1)
    plt.ylabel("Accuracy")
    plt.title("Validation Matched Accuracy")
    plt.legend()
    plt.show()

plot_validation_accuracy(validation_accuracy, target=0.5)